In [1]:
!pip install beir

     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 78 kB 5.5 MB/s 
     |████████████████████████████████| 8.4 MB 16.9 MB/s 
     |████████████████████████████████| 379 kB 49.3 MB/s 
     |████████████████████████████████| 4.9 MB 35.9 MB/s 
     |████████████████████████████████| 3.1 MB 38.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.4 MB/s 
     |████████████████████████████████| 1.2 MB 39.3 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 596 kB 51.6 MB/s 
     |████████████████████████████████| 895 kB 57.2 MB/s 
  Created wheel for beir: filename=beir-0.2.3-py3-none-any.whl size=45805 sha256=a44103c823bb0168edd5bbf481eb6931e9c9d0f12acdc7f9789e8fe4b344422d
  Stored in directory: /root/.cache/pip/wheels/af/29/4c/261c842753d34921b8b35c1b8679720114f8536654767430dd
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp37-cp37m-linux_x86_64.whl size=266803 sha256=bc30228cd4

In [2]:
! pip install datasets transformers

     |████████████████████████████████| 290 kB 2.6 MB/s 
     |████████████████████████████████| 1.1 MB 43.8 MB/s 
     |████████████████████████████████| 132 kB 48.2 MB/s 
     |████████████████████████████████| 243 kB 49.0 MB/s 
     |████████████████████████████████| 192 kB 46.0 MB/s 
     |████████████████████████████████| 160 kB 47.8 MB/s 
     |████████████████████████████████| 271 kB 46.7 MB/s 


In [3]:
# import pandas as pd
# import numpy as np

In [4]:
# from beir import util
# from beir.datasets.data_loader import GenericDataLoader

In [5]:
# from datasets import load_dataset, load_metric

In [6]:
# from typing import List
# from typing import Dict
# from typing import Tuple

In [9]:
from utils_index import *

# Part 1: Adding DBPedia contexts into our SQuADv2 dataset

In [10]:
# Load the dbpedia dataset

dataset = "dbpedia"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip".format(dataset)
data_path = util.download_and_unzip(url, "datasets")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

datasets/dbpedia-entity.zip:   0%|          | 0.00/610M [00:00<?, ?iB/s]

KeyboardInterrupt: ignored

In [11]:
print(inverse(5))

0.2


In [ ]:
# Load the squadv2 dataset

datasets = load_dataset("squad_v2")

In [ ]:
all_good_entries = []

for i in qrels:
  # Check if query is a question, because we want a question-answer system
  # So it feels natural to get contexts related to questions
  if "QALD2" in i:
    # For every entry that correspond to a question
    for j in qrels[i]:
      # Get context
      context = corpus[j]['text']
      # Check if it's long enough for us
      context_tokens = context.split(' ')
      context_words = [word.lower() for word in context_tokens]
      if len(context_words) >= 50 and context not in all_good_entries:
        # Big enough and not yet added into our dataset
        # Add it in the list of good context
        all_good_entries.append(context)

In [ ]:
# Transform the squadv2 validation set into a dictionnary
# To make it easier to manipulate

dico = {}
for i in range(len(datasets['validation'])):
  context = datasets  ['validation'][i]['context']
  if context not in dico:
    new_dico = {}
    new_dico['title'] = datasets['validation'][i]['title']
    new_dico['question'] = datasets['validation'][i]['question']
    new_dico['answers'] = datasets['validation'][i]['answers']
    dico[context] = new_dico

In [ ]:
datasets['validation']

In [ ]:
# Create a list for all questions, contexts, titles and answers
# from our dataset

all_questions = []
all_contexts = []
all_titles = []
all_answers = []

for context in dico.keys():
  all_contexts.append(context)
  all_questions.append(dico[context]['question'])
  all_titles.append(dico[context]['title'])
  all_answers.append(dico[context]['answers'])

In [ ]:
# Add DBPedia context to SQuADv2 dataset

all_contexts = all_contexts + all_good_entries

In [ ]:
len(all_contexts)

In [ ]:
# Put it into a DataFrame 

df = pd.DataFrame({'question': pd.Series(all_questions), 'context': pd.Series(all_contexts), 'title': pd.Series(all_titles), 'answers': pd.Series(all_answers)})
df

In [ ]:
# Collect all unique questions from our dataset

seen = set()
seen_add = seen.add
unique_questions = [x for x in all_questions if not (x in seen or seen_add(x))]
q_a = {}

for question in unique_questions:
  a = df.loc[df['question'] == question]
  l = list(a.index)
  q_a[question] = l

In [ ]:
len(unique_questions)

In [ ]:
unique_questions[:5]

# Partie 2

In [ ]:
# from sentence_transformers import SentenceTransformer
# import sentence_transformers.util

In [ ]:
# def index_model_cosine_similarity(model, questions, contexts):
#   '''
#   Calculate the cosine similarity between given questions
#   and given contexts by using a given model
#   Returns a list of length len(questions) of list of len(contexts)
#   of cosine similarity between question[i] and all contexts
#   '''
#   embeddings = model.encode(contexts, show_progress_bar=True)
#   question_embeddings = model.encode(questions, show_progress_bar=True)
#   similarity = sentence_transformers.util.pytorch_cos_sim(question_embeddings, embeddings)
#   return similarity

# def index_model_dot_product_similarity(model, questions, contexts):
#   '''
#   Calculate the dot product similarity between given questions
#   and given contexts by using a given model
#   Returns a list of length len(questions) of list of len(contexts)
#   of dot product similarity between question[i] and all contexts
#   '''
#   embeddings = model.encode(contexts, show_progress_bar=True)
#   question_embeddings = model.encode(questions, show_progress_bar=True)
#   similarity = sentence_transformers.util.dot_score(question_embeddings, embeddings)
#   return similarity

# def get_ranks(res):
#   '''
#   From the list of list similarities, calculate the reciprocal rank for 
#   each question
#   Returns a list of reciprocal ranks corresponding to a question by their index
#   in the list
#   '''
#   all_ranks = []

#   for i in range(len(res)):
#     similarities = res[i]
#     index = sorted(range(len(similarities)), key=lambda k: similarities[k])
#     index.reverse()
#     question = unique_questions[i]
#     for rank in range(len(a)):
#       if index[rank] in q_a[question]:
#         all_ranks.append(rank + 1)
#         break
#   return all_ranks

# def get_top10_asymetric(similarities):
#   '''
#   Get the top 10 contexts who are the most similar to a 
#   given question.
#   List given as parameter is a list of cosine/dot product between all contexts
#   and a given question
#   '''
#   index = sorted(range(len(similarities)), key=lambda k: similarities[k])
#   index.reverse()
#   return index[:10]

# # To use with map()
# def inverse(n):
#   return 1/n

### Testing different models

In [ ]:
%%time
model_distilbert_v4 = SentenceTransformer('msmarco-distilbert-base-v4')

similarity_distilbert_v4 = index_model_cosine_similarity(model_distilbert_v4, unique_questions, df.context.to_list())
rank_distilbert_v4 = get_ranks(similarity_distilbert_v4)
result_distilbert_v4 = map(inverse, rank_distilbert_v4)
# Compute MRR
print(sum(list(result_distilbert_v4)) / len(rank_distilbert_v4))

In [ ]:
%%time
model_distilbert_v3 = SentenceTransformer('msmarco-distilbert-base-v3')

similarity_distilbert_v3 = index_model_cosine_similarity(model_distilbert_v3, unique_questions, df.context.to_list())
rank_distilbert_v3 = get_ranks(similarity_distilbert_v3)
result_distilbert_v3 = map(inverse, rank_distilbert_v3)
# Compute MRR
print(sum(list(result_distilbert_v3)) / len(rank_distilbert_v3))

In [ ]:
%%time
model_roberta_v3 = SentenceTransformer('msmarco-roberta-base-v3')

similarity_roberta_v3 = index_model_cosine_similarity(model_roberta_v3, unique_questions, df.context.to_list())
rank_roberta_v3 = get_ranks(similarity_roberta_v3)
result_roberta_v3 = map(inverse, rank_roberta_v3)
# Compute MRR
print(sum(list(result_roberta_v3)) / len(rank_roberta_v3))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.70k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/678 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.6636146515360238
CPU times: user 15min 10s, sys: 7.35 s, total: 15min 18s
Wall time: 15min 19s


In [ ]:
%%time
model_distilbert_tas = SentenceTransformer('msmarco-distilbert-base-tas-b')

similarity_distilbert_tas = index_model_dot_product_similarity(model_distilbert_tas, unique_questions, df.context.to_list())
rank_distilbert_tas = get_ranks(similarity_distilbert_tas)
result_distilbert_tas = map(inverse, rank_distilbert_tas)
# Compute MRR
print(sum(list(result_distilbert_tas)) / len(rank_distilbert_tas))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.8203144880079584
CPU times: user 12min 59s, sys: 4.82 s, total: 13min 4s
Wall time: 13min 6s


In [ ]:
%%time
model_roberta_firstp = SentenceTransformer('msmarco-roberta-base-ance-firstp')

similarity_roberta_firstp = index_model_dot_product_similarity(model_roberta_firstp, unique_questions, df.context.to_list())
rank_roberta_firstp = get_ranks(similarity_roberta_firstp)
result_roberta_firstp = map(inverse, rank_roberta_firstp)
# Compute MRR
print(sum(list(result_roberta_firstp)) / len(rank_roberta_firstp))

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/787 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.14k [00:00<?, ?B/s]

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

0.7139793512717401
CPU times: user 15min 17s, sys: 7.07 s, total: 15min 24s
Wall time: 15min 27s


# Part 3

In [ ]:
import faiss
from faiss import normalize_L2

In [ ]:
model_distilbert_tas = SentenceTransformer('msmarco-distilbert-base-tas-b') # best performance overall
embeddings = model_distilbert_tas.encode(df.context.to_list(), show_progress_bar=True)

embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.index.values)

Batches:   0%|          | 0/371 [00:00<?, ?it/s]

In [ ]:
model_distilbert_tas = SentenceTransformer('msmarco-distilbert-base-tas-b') # best performance overall

# def create_index(contexts, model, df):
#   '''
#   Use a sentence-transformer model to encode our contexts, and use
#   faiss to index
#   Returns that index
#   '''
#   embeddings = model.encode(df.context.to_list(), show_progress_bar=True)

#   embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

#   # Index with normalize_L2 to compute the similarities
#   index = faiss.IndexFlatL2(embeddings.shape[1])
#   index = faiss.IndexIDMap(index)
#   index.add_with_ids(embeddings, df.index.values)

#   return index

In [ ]:
# def doc_search(questions, model, index, num_results=10):
#   '''
#   For all questions given as parameters, search for the top k documents 
#   corresponding to a given question
#   Returns a list of distance and index corresponding to a certain context
#   '''
#   vector = model.encode(list(questions), show_progress_bar=True)
#   D, I = index.search(np.array(vector).astype("float32"), k=num_results)
#   return D, I

In [ ]:
# def MMR_test(res):
#   test = []

#   for i in range(len(I)):
#     for rank in range(len(I[i])):
#       if I[i][rank] in q_a[unique_questions[i]]:
#         test.append(rank + 1)
#         break
#   r = map(inverse, test)
#   return sum(list(r)) / len(test))

In [ ]:
%%time
model_distilbert_tas = SentenceTransformer('msmarco-distilbert-base-tas-b') # best performance overall
index = create_index(df.contexts.to_list(), model_distilbert_tas, df)
D, I = vector_search(unique_questions, model_distilbert_tas, index, num_results=10)
test_MRR(I)
# len is everything to measure the MRR
# otherwise, specify 10 for top-10 search

Batches:   0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
# # MRR test

# test = []

# for i in range(len(I)):
#   for rank in range(len(I[i])):
#     if I[i][rank] in q_a[unique_questions[i]]:
#       test.append(rank + 1)
#       break

In [ ]:
# r = map(inverse, test)

In [ ]:
# print(sum(list(r)) / len(test))

0.7869634333920051


In [ ]:
# def get_top10_context(question):
#   list_context = []
#   D, I = vector_search([question], model_distilbert_tas, index, num_results=10)
#   a = df.context.to_list()
#   for i in I[0]:
#     list_context.append(a[i])
#   return list_context
top10 = get_top10_context(unique_questions[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
for i in top10:
  print(i)

The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.
Denmark (/ˈdɛnmɑrk/; Danish: Danmark [ˈd̥ænmɑɡ̊]) is a country in Northern Europe. The southernmost of the Nordic countries, it is located southwest of Sweden and south of Norway, and bordered to the south by Germany. Denmark forms part of the cultural regi